In [79]:
!pip install kagglehub

In [80]:
import pandas as pd
import kagglehub
import os
import re
import numpy as np
import json
import datetime

# Парсинг первого датасета

In [81]:
file_path = kagglehub.dataset_download(
  "thedevastator/uncovering-user-perspectives-on-bitcoin-through")

In [82]:
file_path += '/Bitcoin.csv'

In [89]:
df = pd.read_csv(file_path)

In [90]:
df.columns

Index(['title', 'score', 'id', 'url', 'comms_num', 'created', 'body',
       'timestamp'],
      dtype='object')

In [91]:
df = df[['title', 'score', 'timestamp', 'body']]
df.columns = ['title', 'upvotes', 'date', 'text']
df = df.reindex(columns=df.columns.to_list() + ['upvote_ratio'])

In [92]:
df

,title,upvotes,date,text,upvote_ratio
0,Can someone help explain to me how and why a l...,1,2022-12-19 06:43:02,[https://mempool.space/block/00000000000000000...,NaN
1,Bitcoin is King…,2,2022-12-19 06:19:25,Once you understand #bitcoin and truly underst...,NaN
2,The FTX disaster has set back crypto by ‘years...,0,2022-12-19 05:59:53,As reported by [CNBC](https://www.cnbc.com/202...,NaN
3,Will the loss of more Bitcoins over time resul...,5,2022-12-19 05:53:33,As more people accept Bitcoin and more Bitcoin...,NaN
4,Experts Say Sam Bankman-Fried's Best Legal Def...,21,2022-12-19 05:45:47,NaN,NaN
...,...,...,...,...,...
1892,Comment,2,2022-12-18 23:32:42,I would be LIVID!!!!,NaN
1893,Comment,1,2022-12-18 23:32:34,Homie doesn’t know how the bank works. \nHomie...,NaN
1894,Comment,1,2022-12-18 23:32:22,"He's using muun wallet, wonderful 👍👍",NaN
1895,Comment,3,2022-12-18 23:32:09,"Sure it's not practical, and I think that it's...",NaN


#Парсинг второго датасета

In [93]:
file_path = kagglehub.dataset_download(
  "leukipp/reddit-crypto-data")

In [94]:
file_path

'/root/.cache/kagglehub/datasets/leukipp/reddit-crypto-data/versions/325'

In [98]:
for dirpath, _, filenames in os.walk(file_path):
  for filename in filenames:
    if not re.search('.*\.csv$', filename):
      continue
    df_temp = pd.read_csv(os.path.join(dirpath, filename))
    df_temp = df_temp[['created', 'title', 'is_self', 'upvote_ratio', 'score', 'selftext']]
    df_temp = df_temp[df_temp['is_self']==1]
    df_temp=df_temp.drop(columns=['is_self'])
    df_temp.columns = ['date', 'title', 'upvote_ratio', 'upvotes', 'text']
    df = pd.concat([df, df_temp], ignore_index=True)

In [99]:
df.shape

(530668, 5)

#Парсинг третьего датасета

In [100]:
file_path = kagglehub.dataset_download(
    'pavellexyr/reddit-r-bitcoin-data-for-jun-2022')

In [101]:
file_path

'/root/.cache/kagglehub/datasets/pavellexyr/reddit-r-bitcoin-data-for-jun-2022/versions/1'

In [102]:
file_path += '/reddit-r-bitcoin-data-for-jun-2022-'
df_comments = pd.read_csv(file_path + 'comments.csv')
df_posts = pd.read_csv(file_path + 'posts.csv')

In [103]:
df_comments = df_comments[['created_utc', 'body', 'score']]
df_comments['created_utc'] = df_comments['created_utc'].apply(lambda x : pd.to_datetime(x, utc=True, unit='s'))
df_comments.columns = ['date', 'text', 'upvotes']


In [104]:
df_comments['date']

,date
0,2022-06-30 23:59:53+00:00
1,2022-06-30 23:59:29+00:00
2,2022-06-30 23:59:28+00:00
3,2022-06-30 23:59:24+00:00
4,2022-06-30 23:59:14+00:00
...,...
170027,2022-06-01 00:01:48+00:00
170028,2022-06-01 00:01:43+00:00
170029,2022-06-01 00:00:50+00:00
170030,2022-06-01 00:00:22+00:00


In [105]:
df_posts = df_posts[['selftext', 'title', 'score', 'created_utc']]
df_posts = df_posts[(df_posts['selftext'] != '') &
 (df_posts['selftext'].notnull()) &
  (df_posts['selftext'] != '[deleted]') &
   (df_posts['selftext'] != '[removed]')]
df_posts['created_utc'] = df_posts['created_utc'].apply(lambda x : pd.to_datetime(x, utc=True, unit='s'))
df_posts.columns = ['text', 'title', 'upvotes', 'date']


In [106]:
df_posts

,text,title,upvotes,date
1,I was thinking.. is it possible for the govern...,Bitcoin and the internet,0,2022-06-30 23:42:34+00:00
4,How long is a reasonable amount of time for BT...,Deciphering BTC Hash,0,2022-06-30 23:26:26+00:00
6,I bought a toilet with bitcoin. Why? Because I...,Toilet,21,2022-06-30 23:22:13+00:00
12,"If bitcoins are sent to an invalid address, wh...",Lost coins question,3,2022-06-30 23:14:14+00:00
14,Have had equity in my home for about 14 years ...,Real Estate up/Bitcoin down,2,2022-06-30 23:00:53+00:00
...,...,...,...,...
7516,"I lost my private key, but I have my 12 word p...",Lost It All,8,2022-06-01 03:55:38+00:00
7521,Was the crypto market expected to stay un-coup...,ELI5: Why the de-coupling from the stock marke...,0,2022-06-01 03:08:17+00:00
7526,Let's say you retire with 2 btc...along with o...,what is the correct way to utilize bitcoin as ...,19,2022-06-01 02:20:02+00:00
7530,Still relatively new into my Bitcoin journey (...,How Many HODLers here run a BTC / LN node?,12,2022-06-01 01:40:44+00:00


In [107]:
df = pd.concat([df, df_comments, df_posts], ignore_index=True)

#Сохранение в json

In [108]:
df.shape

(702255, 5)

In [109]:
df.to_json('{}.json'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), orient='records', lines=True)